<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%204/4_7_Ergebnis_und_Auswertung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Installieren aller Pakete

In [ ]:
# Hier die Kaggle Credentials einfügen (ohne Anführungszeichen)

%env KAGGLE_USERNAME=openhpi
%env KAGGLE_KEY=das_ist_der_key

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import img_to_array, array_to_img
from keras.preprocessing.image import img_to_array, array_to_img

from keras.utils.vis_utils import plot_model

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras import layers, models

warnings.simplefilter('always', category=UserWarning)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import rcParams
import string
rcParams['figure.figsize'] = 14, 10

In [ ]:
def show_accuracy_loss_plot(history, num_epochs):
    epochs = [i for i in range(num_epochs)]
    fig , ax = plt.subplots(1,2)
    train_acc = history.history['accuracy']
    train_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    fig.set_size_inches(16,9)
    
    ax[0].plot(epochs , train_acc , 'go-' , label = 'Train Accuracy')
    ax[0].plot(epochs , val_acc , 'ro-' , label = 'Test Accuracy')
    ax[0].set_title('Training & Test Accuracy')
    ax[0].legend()
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("Accuracy")
    
    ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
    ax[1].plot(epochs , val_loss , 'r-o' , label = 'Testing Loss')
    ax[1].set_title('Training & Test Loss')
    ax[1].legend()
    ax[1].set_xlabel("Epochs")
    ax[1].set_ylabel("Loss")
    plt.show()
    
def show_example_pictures(X_train, y_train, alphabetic_label, predictions):
    p = 0
    for i in range(10):
        plt.subplot(3,5,p+1)
        plt.imshow(X_train[i].reshape(28,28), cmap="gray", interpolation='none')
        plt.title("Label: {} Predicted: {}".format(alphabetic_label[y_train[i]], alphabetic_label[predictions[i]]))
        plt.tight_layout()
        p += 1

# 4.7 Ergebnis und Auswertung

<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/cover_sign_language.jpg">



# Einlesen der Daten

In [ ]:
!pip3 install kaggle
!kaggle datasets download -d datamunge/sign-language-mnist

In [ ]:
import zipfile
with zipfile.ZipFile("sign-language-mnist.zip", 'r') as zip_ref:
    zip_ref.extractall("")

In [ ]:
df_train = pd.read_csv("sign_mnist_train.csv")
df_test = pd.read_csv("sign_mnist_test.csv")

In [ ]:
y_test = df_test["label"]
y_train = df_train["label"]
del df_train['label']
del df_test['label']

In [ ]:
# Normalisierung
X_train = df_train.values/255
X_test = df_test.values/255

# Reshaping von 1D zu 3D
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [ ]:
alphabetic_label = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "R", "U", "V", "W", "X", "Y", "Z"]

# Label Encoding

In [ ]:
lb=LabelBinarizer()
lb.fit(y_train)
y_test_oh = lb.transform(y_test)
y_train_oh = lb.transform(y_train)

In [ ]:
data_augmentation = ImageDataGenerator(
                                  rotation_range = 0,
                                  height_shift_range=0.2,
                                  width_shift_range=0.2,
                                  shear_range=0,
                                  horizontal_flip=False,
                                  vertical_flip=False)
data_augmentation.fit(X_train)

In [ ]:
model=Sequential()
model.add(Conv2D(128,kernel_size=(5,5),
                 strides=1,padding='same',activation='relu',input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(3,3),strides=2,padding='same'))
model.add(Conv2D(64,kernel_size=(2,2),
                strides=1,activation='relu',padding='same'))
model.add(MaxPool2D((2,2),2,padding='same'))
model.add(Conv2D(32,kernel_size=(2,2),
                strides=1,activation='relu',padding='same'))
model.add(MaxPool2D((2,2),2,padding='same'))
          
model.add(Flatten())

model.add(Dense(units=256,activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=24,activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(data_augmentation.flow(X_train,y_train_oh,batch_size=128),
         epochs = 40,
          shuffle=1,
           validation_data=(X_test,y_test_oh),
         )

In dieser Einheit wollen wir noch einmal im Detail die Ergebnisse unserer Modelle betrachten. Zunächst betrachten wir noch einmal die Accuracy- und Loss-Kurve für die Trainings- und Testdaten. 

In [ ]:
show_accuracy_loss_plot(history, num_epochs=40)

Wir geben uns erneut einmal die Accuracy unseres Modells direkt aus. 

In [ ]:
(ls,acc)=model.evaluate(x=X_test,y=y_test_oh)
print('Modell Accuracy = {}%'.format(acc*100))

Da wir für zwei Klassen (Klasse 9 - Symbol J und Klasse 25 - Symbol Z) keine Bilder haben, müssen wir unsere Ergebnisse etwas anpassen. Da für Klasse 9 keine Daten vorhanden waren, müssen wir unsere Ergebnis-Labels jeweils um einen Zähler erhöhen, wenn sie an der Stelle >= 9 liegen. Dann stimmen die Labels in unserer Vorhersage mit den realen Labels wieder überein. 

In [ ]:
predictions = np.argmax(model.predict(X_test), axis=-1)
for i in range(len(predictions)):
    if(predictions[i] >= 9):
        predictions[i] += 1

Wir können uns auch jeweils eine detaillierte Analyse der gesamten Klassifikation ausgeben lassen mit einer Auflistung aller Klassen und deren Ergebnisse. Man sieht hierbei, dass z.B. Klasse 1 nicht immer optimale Ergebnisse erzielt. 

In [ ]:
classes = ["Class " + str(i) for i in range(25) if i != 9]
print(classification_report(y_test, predictions, target_names = classes))

Um das Ganze noch etwas graphischer darzustellen, verwenden wir eine sogenannte Confusion Matrix. Hier wird jeweils die tatsächliche Klasse mit der vorhergesagten Klasse verglichen. Stimmen diese überein, dann befinden sich alle Einträge auf der Diagonalen von oben links nach unten rechts. 

In [ ]:
cm = confusion_matrix(y_test,predictions)
cm = pd.DataFrame(cm , index = [i for i in range(25) if i != 9] , columns = [i for i in range(25) if i != 9])

In [ ]:
plt.figure(figsize = (15,15))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='')

Zum Anfang haben wir uns einige Beispielbilder ausgeben lassen. Da wir nun die Vorhersagen gemacht haben, geben wir uns einige Bilder aus mit dem Ergebnis unseres Modells und dem tatsächlichen Label.

In [ ]:
show_example_pictures(X_test, y_test, alphabetic_label, predictions)

Hier wollen wir allerdings einmal festhalten, dass die erreichten Ergebnisse keinesfalls in jedem Datensatz und für jeden Anwendungsfall so gut ausfallen. Das ist grundsätzlich immer Datensatz und Modell spezifisch. Solch gute Ergebnisse lassen sich in realen Anwendungsfällen meist sehr selten erzielen. Wir müssen uns vor Augen führen, dass dieser Datensatz auch unter sehr kontrollierten Bedingungen aufgenommen wurde. Die Symbole wurden jeweils vor einem neutralen Hintergrund aufgenommen. 

Wir man diese Bildanalyse in der Praxis einsetzen, so wird man sicherlich auf einige Probleme stoßen (z.B. bei verschiedenen Hintergründen, durch Bewegung verzerrte Bilder). 

Das war es zu unserer Einheit **Ergebnis und Auswertung** für den Anwendungsfall der Bilderkennung von Gebärdensprache.